<a href="https://colab.research.google.com/github/whispermy/novatus_academia/blob/main/Week3_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AI 노바투스과정**

Week 3: 딥러닝 기초 I (2021-08-20)

실습 #3: 신경망 모형의 초모수 결정하기/ Early-Stopping 실습하기

## 1. 학습률 스케줄링 하기



In [6]:
#keras 라이브러리 가져오기

import tensorflow.keras as keras

In [7]:
#MNIST 패션 데이터 로딩하기

fashion_mnist=keras.datasets.fashion_mnist

In [8]:
(X_train_full, y_train_full), (X_test, y_test) =fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [9]:
#target 의 각 숫자값에 해당하는 클래스 명 지정하기

class_names=["T-shirt/top",'Trouser','Pullover','Dress',"Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [10]:
# 학습용 데이터를 다시 학습용 데이터와 검증용 데이터 (valdiation set) 으로 분리
# 분리와 동시에 정규화

X_valid, X_train= X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train= y_train_full[:5000], y_train_full[5000:]

X_test=X_test/255.0

In [11]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


### 1.1 power scheduling

In [15]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)
### power scheduling 으로 학습률을 스케줄링하기 !!  

optimizer = keras.optimizers.SGD(lr=0.01, decay = 0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [16]:
#epoch 개수를 30개로 설정하여 모형 학습하기

history=model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.7282 - accuracy: 0.7602 - val_loss: 0.5078 - val_accuracy: 0.8262
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4911 - accuracy: 0.8284 - val_loss: 0.4755 - val_accuracy: 0.8364
Epoch 3/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4481 - accuracy: 0.8425 - val_loss: 0.4356 - val_accuracy: 0.8494
Epoch 4/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4247 - accuracy: 0.8516 - val_loss: 0.4242 - val_accuracy: 0.8522
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4069 - accuracy: 0.8570 - val_loss: 0.4084 - val_accuracy: 0.8592
Epoch 6/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3935 - accuracy: 0.8618 - val_loss: 0.3963 - val_accuracy: 0.8632
Epoch 7/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3828 - accuracy: 0.8665 - val_loss: 0.3849 - val_accuracy:

In [17]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3676 - accuracy: 0.8678


[0.36756521463394165, 0.8677999973297119]

### 1.2 Exponential scheduling

In [18]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [19]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [20]:
#callback 으로 scheduler 지정하기

def exponential_decay_fn(epoch,lr):
  return lr*0.1**(1/20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)


In [22]:
#모형 학습하기

model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[lr_scheduler])

Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2575 - accuracy: 0.9079 - val_loss: 0.3114 - val_accuracy: 0.8860
Epoch 2/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2510 - accuracy: 0.9094 - val_loss: 0.3021 - val_accuracy: 0.8914
Epoch 3/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2452 - accuracy: 0.9123 - val_loss: 0.3041 - val_accuracy: 0.8890
Epoch 4/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2396 - accuracy: 0.9141 - val_loss: 0.2977 - val_accuracy: 0.8920
Epoch 5/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2359 - accuracy: 0.9150 - val_loss: 0.3062 - val_accuracy: 0.8894
Epoch 6/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2305 - accuracy: 0.9177 - val_loss: 0.2955 - val_accuracy: 0.8952
Epoch 7/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2273 - accuracy: 0.9191 - val_loss: 0.2906 - val_accuracy:

In [23]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3180 - accuracy: 0.8877


[0.31795746088027954, 0.8877000212669373]

### 1.3 Performance scheduling

In [24]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [25]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [26]:
#callback 으로 scheduler 지정하기

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)



In [28]:
# 모형 학습하기

model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[lr_scheduler])

Epoch 1/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.7017 - accuracy: 0.7717 - val_loss: 0.5150 - val_accuracy: 0.8240
Epoch 2/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4895 - accuracy: 0.8291 - val_loss: 0.4976 - val_accuracy: 0.8206
Epoch 3/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4442 - accuracy: 0.8451 - val_loss: 0.4269 - val_accuracy: 0.8494
Epoch 4/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4160 - accuracy: 0.8536 - val_loss: 0.4079 - val_accuracy: 0.8600
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3959 - accuracy: 0.8604 - val_loss: 0.3867 - val_accuracy: 0.8634
Epoch 6/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3810 - accuracy: 0.8649 - val_loss: 0.3799 - val_accuracy: 0.8674
Epoch 7/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3672 - accuracy: 0.8708 - val_loss: 0.3680 - val_accuracy:

In [29]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3257 - accuracy: 0.8858


[0.32569625973701477, 0.8858000040054321]

## 2. Early-stopping 수행하기

### 2.1 ModelCheckpoint callback 을 이용하는 방법

In [30]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [31]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [32]:
#callback 만들기

checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

In [33]:
#모형 학습하기

model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb])
# model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, lr_scheduler]
# -> 학습시킬때 같이 복합적으로 하고 싶으면 적어준다

Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7330 - accuracy: 0.7591 - val_loss: 0.5304 - val_accuracy: 0.8260
Epoch 2/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4947 - accuracy: 0.8278 - val_loss: 0.4542 - val_accuracy: 0.8500
Epoch 3/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4491 - accuracy: 0.8438 - val_loss: 0.4799 - val_accuracy: 0.8364
Epoch 4/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4207 - accuracy: 0.8535 - val_loss: 0.4002 - val_accuracy: 0.8622
Epoch 5/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4005 - accuracy: 0.8596 - val_loss: 0.4195 - val_accuracy: 0.8490
Epoch 6/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3841 - accuracy: 0.8650 - val_loss: 0.3803 - val_accuracy: 0.8694
Epoch 7/30
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3699 - accuracy: 0.8687 - val_loss: 0.3592 - val_accuracy:

In [34]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3359 - accuracy: 0.8841


[0.33594951033592224, 0.8841000199317932]

In [ ]:
#저장된 모형 불러오기



### 2.2 EarlyStopping callback 을 이용하는 방법

In [35]:
# keras 의 Sequential API 를 활용하여 신경망 모형의 골격 만들기 
# 은닉층이 2개이며 첫번째 은닉층에 뉴런 300개, 두번째 은닉층에 뉴런 100개 인 골격 만들기
# 활성함수는 relu 사용하기 


model=keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))

model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [36]:
# 모형 컴파일하기 (손실함수, 최적화 방법 지정하기)

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])


In [37]:
#callback 만들기

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=5, restore_best_weights=True)
# keras.callbacks.EarlyStopping(monitor="accuracy",patience=5, restore_best_weights=True)
# -> train data에 대해서 할려면 그냥 accuracy로 기입


In [38]:
#모형 학습하기

model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[early_stopping_cb])


Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7276 - accuracy: 0.7606 - val_loss: 0.5286 - val_accuracy: 0.8166
Epoch 2/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4929 - accuracy: 0.8282 - val_loss: 0.4367 - val_accuracy: 0.8522
Epoch 3/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4464 - accuracy: 0.8447 - val_loss: 0.4312 - val_accuracy: 0.8502
Epoch 4/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4178 - accuracy: 0.8520 - val_loss: 0.4064 - val_accuracy: 0.8620
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3970 - accuracy: 0.8605 - val_loss: 0.3832 - val_accuracy: 0.8686
Epoch 6/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3807 - accuracy: 0.8649 - val_loss: 0.3697 - val_accuracy: 0.8708
Epoch 7/30
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3670 - accuracy: 0.8687 - val_loss: 0.3608 - val_accuracy:

In [39]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3318 - accuracy: 0.8804


[0.3318115472793579, 0.8804000020027161]